In [ ]:
import random
import os

import faiss
import pickle
import numpy as np
from sklearn.decomposition import PCA

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%cd ..

In [4]:
def get_memory(index):
    # write index to file
    faiss.write_index(index, './temp.index')
    # get file size
    file_size = os.path.getsize('./temp.index')
    # delete saved index
    os.remove('./temp.index')
    print(f"File size: {file_size/1024**2} MB")

# Setting up the data

In [107]:
feature_list = pickle.load(open('features/features-caltech101-resnet.pickle','rb'))
# feature_list = feature_list/np.linalg.norm(feature_list,axis=1).reshape(-1,1)  # normalize features so each column has length 1
pca = PCA(n_components=128)
feature_list_compressed = pca.fit_transform(feature_list)


# Flat (Ground Truth)

In [785]:
index = faiss.IndexFlatL2(2048)
index.train(feature_list)
index.add(feature_list)

index_compressed = faiss.IndexFlatL2(128)
index_compressed.train(feature_list_compressed)
index_compressed.add(feature_list_compressed)

get_memory(index)
get_memory(index_compressed)

File size: 71.43754291534424 MB
File size: 4.464886665344238 MB


# IVFFlat

In [786]:
index_ivf = faiss.index_factory(2048, 'IVF100,Flat')
index_ivf.train(feature_list)
index_ivf.add(feature_list)

index_ivf_compressed = faiss.index_factory(128, 'IVF100,Flat')
index_ivf_compressed.train(feature_list_compressed)
index_ivf_compressed.add(feature_list_compressed)


get_memory(index_ivf)
get_memory(index_ivf_compressed)

File size: 72.28940868377686 MB
File size: 4.5843305587768555 MB


# IVFPQ
- 30x memory savings compared to IVFFlat

In [816]:
nlist = 100
m = 8
nbits=8

quantizer = faiss.IndexFlatL2(2048)  # this remains the same
index_ivfpq = faiss.IndexIVFPQ(quantizer, 2048, nlist, m, nbits)
# index_ivfpq = faiss.index_factory(2048, 'IVF100,PQ8')
index_ivfpq.train(feature_list)
index_ivfpq.add(feature_list)

quantizer = faiss.IndexFlatL2(128)  # this remains the same
index_ivfpq_compressed = faiss.IndexIVFPQ(quantizer, 128, nlist, m, nbits)
# index_ivfpq_compressed = faiss.index_factory(128, 'IVF100,PQ8')
index_ivfpq_compressed.train(feature_list_compressed)
index_ivfpq_compressed.add(feature_list_compressed)

get_memory(index_ivfpq)  
get_memory(index_ivfpq_compressed)

WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 9144 points to 256 centroids: please prov

File size: 2.921710968017578 MB
File size: 0.3142890930175781 MB


WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points


# OPQIVFPQ
- Extremely slow for large dimensions  (https://github.com/facebookresearch/faiss/issues/625)
- Interrupted kernel at 4 mins for 2048 dimensions, only attempting 128 dimensions for OPQ
- Slightly larger index than IVFPQ

In [788]:
index_opqivfpq_compressed = faiss.index_factory(128, 'OPQ8,IVF100,PQ8')
index_opqivfpq_compressed.train(feature_list_compressed)
index_opqivfpq_compressed.add(feature_list_compressed)

get_memory(index_opqivfpq_compressed) 

WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 9144 points to 256 centroids: please prov

File size: 0.37685680389404297 MB


In [789]:
random_image_indices = random.sample(range(0, len(feature_list)), 100)
random_feature_list = np.array([feature_list[each_index] for each_index in random_image_indices])
random_feature_list_compressed = np.array([feature_list_compressed[each_index] for each_index in random_image_indices])

# Ground truth distances

In [811]:
D_flat, I_flat = index.search(random_feature_list, 6)
D_flat_compressed, I_flat_compressed = index_compressed.search(random_feature_list_compressed, 5)

# Comparing indexes against ground truth
- IVF 100 is very high, close to known number of clusters at ~101, which helps recall
- IVFFlat is almost same size as IVFFlat (compressed) but slightly higher recall
- Strangely IVFPQ compressed from 2048 to 128 dimensions increases recall@5 0.44 to 0.66
- When trying to improve IVFPQ, lowering nbits from 8 to 7 to address the warnings leaves exact same 0.44 recall on IVFPQ and worsens IVFPQ compressed from 0.66 to 0.6 -> may be good to ignore warnings
- For this dataset, IVFPQ has 30x smaller memory footprint than IVFFlat but only slightly lower recall 0.66 vs 0.79

In [812]:
def compare(I_gt,feature_list,index, name=''):
    D, I = index.search(feature_list, 6)
    print(f'Recall@5 for {name}: {(I[:,1:6] == I_gt[:,[1]]).sum()/len(feature_list)}')

In [817]:
index_names = ['IVFFlat','IVFPQ']
index_names_compressed = ['IVFFlat (compressed)','IVFPQ (compressed)','OPQIVFPQ (compressed)']

indexes = [index_ivf,
           index_ivfpq]

indexes_compressed = [index_ivf_compressed,
                      index_ivfpq_compressed,
                      index_opqivfpq_compressed]

for name, idx in zip(index_names, indexes):
    compare(I_flat,random_feature_list,idx, name)
    
    
for name, idx in zip(index_names_compressed, indexes_compressed):
    compare(I_flat_compressed,random_feature_list_compressed,idx, name)

Recall@5 for IVFFlat: 0.8
Recall@5 for IVFPQ: 0.44
Recall@5 for IVFFlat (compressed): 0.79
Recall@5 for IVFPQ (compressed): 0.66
Recall@5 for OPQIVFPQ (compressed): 0.71


## Checking what lowers recall

In [822]:
index_ivf_compressed = faiss.index_factory(128, 'IVF100,PQ8')
index_ivf_compressed.train(feature_list_compressed)
index_ivf_compressed.add(feature_list_compressed)

compare(I_flat_compressed,random_feature_list_compressed,index_ivf_compressed)

WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points


Recall@5 for : 0.66


## Experiments
- Lower nlist 100 to 50
- Lower PQ M 8 to 2

Results
- nlist of IVF has much larger impact on recall@5 than nbits of PQ


In [823]:
index_ivfpq_compressed = faiss.index_factory(128, 'IVF100,PQ2')
index_ivfpq_compressed.train(feature_list_compressed)
index_ivfpq_compressed.add(feature_list_compressed)

compare(I_flat_compressed,random_feature_list_compressed,index_ivfpq_compressed)

index_ivfpq_compressed = faiss.index_factory(128, 'IVF50,PQ8')
index_ivfpq_compressed.train(feature_list_compressed)
index_ivfpq_compressed.add(feature_list_compressed)

compare(I_flat_compressed,random_feature_list_compressed,index_ivfpq_compressed)

index_ivfpq_compressed = faiss.index_factory(128, 'IVF50,PQ2')
index_ivfpq_compressed.train(feature_list_compressed)
index_ivfpq_compressed.add(feature_list_compressed)

compare(I_flat_compressed,random_feature_list_compressed,index_ivfpq_compressed)

WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points


Recall@5 for : 0.42


WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points


Recall@5 for : 0.64


WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points


Recall@5 for : 0.36


# Custom IndexPQ

In [10]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import euclidean_distances

class CustomIndexPQ:
    
    BITS2DTYPE = {
        8: np.uint8,
        16: np.uint16,
    }
    
    def __init__(self,d: int,m: int,nbits: int,) -> None:
        """Custom IndexPQ implementation.
        Parameters
        ----------
        d
            Dimensionality of the original vectors.
        m
            Number of segments.
        nbits
            Number of bits.
        """
        if d % m != 0:
            raise ValueError("d needs to be a multiple of m")

        if nbits not in CustomIndexPQ.BITS2DTYPE:
            raise ValueError(f"Unsupported number of bits {nbits}")

        self.m = m
        self.k = 2**nbits
        self.d = d
        self.ds = d // m

        self.estimators = [KMeans(n_clusters=self.k, random_state=1) for _ in range(m)]

        self.is_trained = False

        self.dtype = CustomIndexPQ.BITS2DTYPE[nbits]
        self.dtype_orig = np.float32

    def train(self, X: np.ndarray) -> None:
        """Train M KMeans estimators.
        Parameters
        ----------
        X
            Array of shape `(n, d)` and dtype `float32`.
        """
        for i in range(self.m):
            estimator = self.estimators[i]
            X_i = X[:, i * self.ds : (i + 1) * self.ds]
            estimator.fit(X_i)

        self.is_trained = True


    def encode(self, X: np.ndarray) -> np.ndarray:
        """Encode original features into codes.
        Parameters
        ----------
        X
            Array of shape `(n_queries, d)` of dtype `np.float32`.
        Returns
        -------
        result
            Array of shape `(n_queries, m)` of dtype `np.uint8`.
        """
    
        n = len(X)
        result = np.empty((n, self.m), dtype=self.dtype)  #Prevents automatic 'float64' causing IndexError: arrays used as indices must be of integer (or boolean) type

        for i in range(self.m):
            estimator = self.estimators[i]
            X_i = X[:, i * self.ds : (i + 1) * self.ds]
            result[:, i] = estimator.predict(X_i)

        return result

    def add(self, X: np.ndarray) -> None:
        """Add vectors to the database (their encoded versions).
        Parameters
        ----------
        X
            Array of shape `(n_codes, d)` of dtype `np.float32`.
        """
        if not self.is_trained:
            raise ValueError("The quantizer needs to be trained first.")
        self.codes = self.encode(X)

    def compute_asymmetric_distances(self, X: np.ndarray) -> np.ndarray:
        """Compute asymmetric distances to all database codes.
        Parameters
        ----------
        X
            Array of shape `(n_queries, d)` of dtype `np.float32`.
        Returns
        -------
        distances
            Array of shape `(n_queries, n_codes)` of dtype `np.float32`.
        """
        if not self.is_trained:
            raise ValueError("The quantizer needs to be trained first.")

        if self.codes is None:
            raise ValueError("No codes detected. You need to run `add` first")

        n_queries = len(X)
        n_codes = len(self.codes)

        distance_table = np.empty(
            (n_queries, self.m, self.k), dtype=self.dtype_orig
        )  # (n_queries, m, k)

        for i in range(self.m):
            X_i = X[:, i * self.ds : (i + 1) * self.ds]  # (n_queries, ds)
            centers = self.estimators[i].cluster_centers_  # (k, ds)
            distance_table[:, i, :] = euclidean_distances(X_i, 
                                                          centers, 
                                                          squared=True)

        distances = np.zeros((n_queries, n_codes), dtype=self.dtype_orig)

        for i in range(self.m):
            distances += distance_table[:, i, self.codes[:, i]]

        return distances

    def search(self, X: np.ndarray, k: int) -> tuple:
        """Find k closest database codes to given queries.
        Parameters
        ----------
        X
            Array of shape `(n_queries, d)` of dtype `np.float32`.
        k
            The number of closest codes to look for.
        Returns
        -------
        distances
            Array of shape `(n_queries, k)`.
        indices
            Array of shape `(n_queries, k)`.
        """
        n_queries = len(X)
        distances_all = self.compute_asymmetric_distances(X)

        indices = np.argsort(distances_all, axis=1)[:, :k]

        distances = np.empty((n_queries, k), dtype=np.float32)
        for i in range(n_queries):
            distances[i] = distances_all[i][indices[i]]

        return distances, indices

In [11]:
d, m, nbits = 128,8,8
custom = CustomIndexPQ(d, m, nbits)

In [12]:
custom.train(feature_list_compressed)
custom.add(feature_list_compressed)

/Users/hanqi/code/ahrefs/venv/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Users/hanqi/code/ahrefs/venv/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Users/hanqi/code/ahrefs/venv/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Users/hanqi/code/ahrefs/venv/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.wa

In [827]:
D_custom, I_custom = custom.search(random_feature_list_compressed, 5)
D_custom[:5], I_custom[:5]

(array([[10527.352 , 13998.237 , 15301.726 , 15498.508 , 15772.971 ],
        [ 9672.746 , 12653.403 , 12746.408 , 12782.03  , 13135.024 ],
        [ 8957.763 , 13945.396 , 14109.893 , 14781.04  , 14956.603 ],
        [14499.895 , 16428.25  , 17053.215 , 21339.955 , 23001.535 ],
        [11837.461 , 13613.828 , 14375.717 , 14442.3955, 14513.7705]],
       dtype=float32),
 array([[8115, 8131, 8156, 8166, 8117],
        [7977, 7983, 8028, 8003, 8001],
        [ 583,  586,  582,  707,  588],
        [7474, 7472, 7473, 7483, 7503],
        [5807, 5803, 5846, 5849, 5865]]))

# Comparison of CustomIndexPQ with faiss IndexPQ
- Slightly different neighbors but generally the same results

In [828]:
d, m, nbits = 128, 8, 8
index_pq_compressed = faiss.IndexPQ(d, m, nbits)
index_pq_compressed.train(feature_list_compressed)
index_pq_compressed.add(feature_list_compressed)
D, I = index_pq_compressed.search(random_feature_list_compressed, 5)

WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points


In [829]:
D[:5],I[:5]

(array([[12010.435, 13992.476, 14116.026, 14533.341, 15198.48 ],
        [ 9567.162, 11588.168, 12361.848, 13200.58 , 13280.393],
        [ 8834.732, 14526.405, 14526.405, 14662.773, 14812.835],
        [16673.625, 18272.426, 19130.832, 21897.4  , 22467.85 ],
        [13159.116, 15042.82 , 15119.03 , 16003.898, 16171.402]],
       dtype=float32),
 array([[8115, 8141, 8108, 8154, 8136],
        [7977, 8002, 8027, 8028, 8006],
        [ 583,  580, 1015,  703,  587],
        [7474, 7472, 7473, 7515, 7500],
        [5807, 5849, 5846, 5865, 5847]]))

In [830]:
compare(I_flat_compressed,random_feature_list_compressed,index_pq_compressed)
compare(I_flat_compressed,random_feature_list_compressed,custom)

Recall@5 for : 0.67
Recall@5 for : 0.67


# Using more difficult VOC2012 data
- Custom code does worse than faiss 0.06 vs 0.13

In [831]:
voc_feature_list = pickle.load(open('features/features-voc2012-resnet.pickle','rb'))
pca = PCA(n_components=128)
voc_feature_list_compressed = pca.fit_transform(voc_feature_list)

In [832]:
random_image_indices = random.sample(range(0, len(voc_feature_list)), 100)
voc_random_feature_list = np.array([voc_feature_list[each_index] for each_index in random_image_indices])
voc_random_feature_list_compressed = np.array([voc_feature_list_compressed[each_index] for each_index in random_image_indices])

In [835]:
#Ground truth
D_flat_compressed, I_flat_compressed = index_compressed.search(voc_random_feature_list_compressed, 6)

In [836]:
compare(I_flat_compressed,voc_random_feature_list_compressed,index_pq_compressed)
compare(I_flat_compressed,voc_random_feature_list_compressed,custom)

Recall@5 for : 0.13
Recall@5 for : 0.06


# Custom IndexIVFPQ
- Explanation of PQ and IVFPQ at https://lear.inrialpes.fr/pubs/2011/JDS11/jegou_searching_with_quantization.pdf

**Differences between my implementation and paper**
- My inverted index did not have `{coarse_centroid: [(id1, code1),...]}` structure like in paper, but more simply `{coarse_centroid:[id1,id2,...]}` and another array of codes because this data model is easier to handle (need to learn C++, SIMD first to appreciate the storage patterns for high performance compute)

**Problems**
- Kmeans coarse clustering too slow (36 secs for 9k images, 128 dim), there must be faster way
- Search over 9k images, 128 dim takes 20 seconds, still slow


## Train and Add

### Creating coarse quantizer and residual database vectors

In [35]:
def fit_coarse_quantizer(feature_list):
    d = 128 # dimension
    m=8
    nlist = 100
    nbits = 8
    k=2**nbits

    coarse_quantizer = KMeans(n_clusters=nlist, random_state=1).fit(feature_list)
    feature_list_residual = feature_list - coarse_quantizer.cluster_centers_[coarse_quantizer.labels_]  # generate residual database vectors to be fine quantized
    return feature_list_residual, coarse_quantizer

feature_list_residual, coarse_quantizer = fit_coarse_quantizer(feature_list_compressed)

/Users/hanqi/code/ahrefs/venv/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [1231]:
def apply_coarse_quantizer(feature_list):
    """Find closest IVF centroid using coarse quantizer and get residuals"""
    ivf_cells = coarse_quantizer.predict(feature_list)
    feature_list_residual = feature_list - coarse_quantizer.cluster_centers_[ivf_cells]  # generate residual database vectors to be fine quantized
    return feature_list_residual, coarse_quantizer, ivf_cells

feature_list_residual, coarse_quantizer, ivf_cells = apply_coarse_quantizer(feature_list_compressed)

In [1232]:
def fit_fine_quantizer(feature_list_residual, d, m):
    ds = d//m

    fine_quantizers = [KMeans(n_clusters=k, random_state=1) for _ in range(m)]

    for i in range(m):
        X_i = feature_list_residual[:, i * ds : (i + 1) * ds]
        fine_quantizers[i].fit(X_i)
    return fine_quantizers

fine_quantizers = fit_fine_quantizer(feature_list_residual, 128, 8)

/Users/hanqi/code/ahrefs/venv/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Users/hanqi/code/ahrefs/venv/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Users/hanqi/code/ahrefs/venv/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Users/hanqi/code/ahrefs/venv/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.wa

### Product Quantizing residuals into codes

In [1233]:
def quantize_residuals(feature_list_residual, estimators, m): 
    n, d= feature_list_residual.shape
    ds = d//m
    codes = np.empty((n, m), dtype=np.uint8)  #Prevents automatic 'float64' causing IndexError: arrays used as indices must be of integer (or boolean) type

    for i in range(m):
        estimator = fine_quantizers[i]
        X_i = feature_list_residual[:, i * ds : (i + 1) * ds]
        codes[:, i] = estimator.predict(X_i)  # shape n number of vectors, m segments
        
    return codes

codes = quantize_residuals(feature_list_residual, fine_quantizers, m)

### Assigning residuals to inverted list

In [1234]:
max_id = 0
codes_db = np.empty((0,m),dtype=np.int8) # always cleared in jupyter for convenience

In [1246]:
def add_inverted_list(ivf_cells, codes, codes_db):
    from collections import defaultdict

    inverted_list = defaultdict(list)

    for idx, coarse_center in enumerate(ivf_cells):  
        inverted_list[coarse_center].append(idx)
    
    codes_db = np.vstack([codes_db, codes])
    
    return inverted_list

inverted_list = add_inverted_list(ivf_cells, codes, codes_db)

# useful for checking correct number of candidates got extracted from probing during search
for key, value in inverted_list.items():
    print(key, len(value))

1 103
93 162
87 113
77 113
3 98
25 97
83 52
28 84
81 52
49 130
94 150
52 70
65 103
12 91
75 68
43 122
98 78
4 121
6 49
40 115
62 118
9 132
99 60
42 87
96 67
97 51
21 51
64 45
7 104
27 117
84 105
41 120
32 54
35 76
67 183
34 48
14 144
86 124
46 76
72 89
33 70
29 40
10 174
85 65
55 103
91 39
73 85
38 57
20 104
50 88
95 78
89 161
79 141
2 113
66 46
78 43
24 89
23 97
44 108
30 129
5 151
37 135
69 115
74 87
57 125
82 112
92 116
8 166
15 153
58 148
18 69
60 53
17 101
70 92
68 46
26 56
13 85
16 123
39 68
48 53
51 73
53 106
88 95
47 65
71 46
0 71
56 92
76 67
61 55
36 43
19 120
63 67
22 94
54 41
90 37
31 74
80 62
45 96
11 70
59 64


## Search

In [1236]:
query = feature_list_compressed[[0]] #  2d to because euclidean_distances expects that
query
nprobe = 1

array([[ 3.1867094e+01, -9.8064718e+00, -1.1358043e+01, -2.0601606e+01,
         7.6451123e-01,  2.9910746e+01, -1.1899613e+01, -1.1767055e+01,
         1.0866048e+01,  6.0190296e+01,  1.4909669e+00,  5.9952347e+01,
        -1.3209792e+01, -4.8219257e+01, -2.7503124e+01, -9.1865454e+00,
        -9.0401993e+00,  7.2042956e+00, -2.8980122e+01,  6.1889820e+00,
         3.3465500e+01, -3.0763933e+01,  7.2214108e+00,  2.8699830e+00,
         7.2017416e-02, -1.1203322e+01,  1.0475516e+01, -1.0316180e+01,
        -1.2320327e+01,  1.3685087e+01,  3.2958090e+00,  1.9592484e+00,
        -8.7479895e-01, -1.2692191e+01, -9.5121059e+00, -1.8125988e+01,
        -9.3066320e+00, -5.9284525e+00,  1.7693888e+01,  1.6936089e+01,
         3.7598002e+00, -7.9874825e+00, -1.5846498e+01,  7.4989166e+00,
         5.4958344e+00,  4.6651011e+00, -2.8053744e+00, -2.7807868e+00,
         3.2602324e+00,  1.4717191e+00,  2.6552942e+00, -9.8519087e+00,
        -8.2460146e+00, -1.1753324e+01, -7.9538870e+00, -8.17695

### Calculate distance of query vector (not residual) to all coarse centroids

In [1237]:
def distance_to_IVFcentroids(query, coarse_quantizer, nprobe):
    query_distance_to_coarse_centroids = euclidean_distances(query, coarse_quantizer.cluster_centers_, squared=True)[0]
    nearest_inverted_keys = np.argsort(query_distance_to_coarse_centroids)[:nprobe]  # argsort gives index of closest coarse centroids, to be filtered by nprobe
    return nearest_inverted_keys 

nearest_inverted_keys = distance_to_IVFcentroids(query, coarse_quantizer, nprobe)

### Testing calculations in 1 cell (assuming nprobe=1)

### Generating query residual vector

In [1238]:
print('Closest coarse centroid: ',nearest_inverted_keys[0])

def generate_query_residual(query, coarse_quantizer, current_cell):
      # closest from coarse quantizer
    query_residual = query - coarse_quantizer.cluster_centers_[current_cell]  # generate residual query to be compared against all quantized residuals
    # print(query_residual.shape, query_residual[:3])
    
    return query_residual

current_cell = nearest_inverted_keys[0]
query_residual = generate_query_residual(query, coarse_quantizer, current_cell)
    

Closest coarse centroid:  1


### Generating distance table

In [1239]:
def compute_distance_table(query_residual, fine_quantizers):
    distance_table = np.empty((m, k), dtype=np.float32)  # shape m segments, distance to k clusters 

    d = query_residual.shape[1]
    ds = d//m
    for i in range(m):
        X_i = query_residual[:, i * ds : (i + 1) * ds]
        centers = fine_quantizers[i].cluster_centers_  # (k, ds)
        distance_table[i, :] = euclidean_distances(X_i, centers, squared=True)
    return distance_table
distance_table = compute_distance_table(query_residual, fine_quantizers)

### Filtering residual vectors using inverted list

In [1251]:
def filter_residual_vectors(inverted_list, codes, current_cell):
    
    filtered_ids = inverted_list[current_cell]
    filtered_result = codes[filtered_ids]
    # print(filtered_result.shape, filtered_result[:3])
    
    return filtered_result, filtered_ids

current_cell = nearest_inverted_keys[0]

filtered_result, filtered_ids = filter_residual_vectors(inverted_list, codes, current_cell)

### Calculating distances on filtered residual vectors

In [1252]:
def calculate_distances(filtered_result, distance_table):
    distances = np.zeros(len(filtered_result), dtype=np.float32)

    for i in range(m):
        distances += distance_table[i, filtered_result[:, i]]
    # print(distances.shape, distances[:3])
    
    return distances

distances = calculate_distances(filtered_result, distance_table)


In [1253]:
def find_smallest_k(distances, filtered_ids, k_nearest):
    import heapq
    import operator

    distance_id = zip(distances,filtered_ids)
    D, I = zip(*heapq.nsmallest(k_nearest, distance_id, operator.itemgetter(0)))
    
    return D, I

k_nearest = 6
D, I = find_smallest_k(distances, filtered_ids, k_nearest)   # generators can only be used once! Re-run previous cell to regenerate distance_id if needed
D, I

((8188.985, 15658.488, 16366.43, 17327.34, 17658.04, 17810.049),
 (0, 25, 168, 470, 229, 364))

### Repeat above steps for all query vectors, and all nprobe

In [1279]:
def search(query, 
                  coarse_quantizer,
                  nprobe,
                  codes,
                  k_nearest
                  ):
    
    nearest_inverted_keys = distance_to_IVFcentroids(query, coarse_quantizer, nprobe)
    
    nprobe_distances = np.array([], dtype=np.float32)
    nprobe_filtered_ids = np.array([], dtype=np.uint64)
    
    for current_cell in nearest_inverted_keys:
        query_residual = generate_query_residual(query, coarse_quantizer, current_cell)
        distance_table = compute_distance_table(query_residual, fine_quantizers)
        filtered_result, filtered_ids = filter_residual_vectors(inverted_list, codes, current_cell)
        distances = calculate_distances(filtered_result, distance_table)
        nprobe_distances = np.append(nprobe_distances, distances)
        nprobe_filtered_ids = np.append(nprobe_filtered_ids, filtered_ids)
        
    D, I = find_smallest_k(nprobe_distances, nprobe_filtered_ids, k_nearest)
    
    return D, I

## Testing how recall varies with nprobe

In [1282]:
k_nearest = 6
m, k = 8, 2**nbits

nprobe_test = {}
# nprobes_to_test = range(1,4)
nprobes_to_test = [1]

for nprobe in nprobes_to_test:
    D_list = []
    I_list = []
    for query in feature_list_compressed:
        D, I = search(query.reshape(1,-1),   #sklearn euclidean_distances needs 2D
                            coarse_quantizer,
                            nprobe,
                            codes,
                            k_nearest)
        D_list.append(D)
        I_list.append(I)
        
    D_list = np.array(D_list, dtype=np.float32)
    I_list = np.array(I_list, dtype=np.uint64)
    
    nprobe_test[nprobe] = (D_list, I_list)

In [1283]:
I_list = nprobe_test[1][1]
print(f'Recall@5 for Custom IndexIVFPQ nprobe={nprobe}:', (I_list[:,1:6] == I_flat[:,[1]]).sum()/len(feature_list_compressed))

Recall@5 for Custom IndexIVFPQ nprobe=1: 0.6917104111986002


## Comparing Custom IVFPQ with faiss

In [146]:
import faiss

d = 128 # dimension
nlist = 100
m=8
nbits = 8

quantizer = faiss.IndexFlatL2(d)  
index = faiss.IndexIVFPQ(quantizer, d, nlist, m, nbits)
                                  
index.train(feature_list_compressed)
index.add(feature_list_compressed)                        

WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points


In [147]:
k_nearest = 5
D, I = index.search(feature_list_compressed, k_nearest)
D[:5], I[:5]

(array([[ 8574.242 , 15943.098 , 16497.477 , 16979.555 , 17299.773 ],
        [10805.41  , 18739.096 , 19096.94  , 20095.824 , 20404.768 ],
        [ 2738.3652,  4321.1904,  4330.2686,  4420.0005,  4439.327 ],
        [ 7315.83  , 12940.91  , 13020.773 , 14139.178 , 15500.04  ],
        [10180.457 , 18115.746 , 18790.076 , 20177.752 , 20539.121 ]],
       dtype=float32),
 array([[   0,  168,    8,  364,   25],
        [   1,  213,   54, 7960,   83],
        [   2,  226,  431,  401,  306],
        [   3,  208,   75, 6222, 3499],
        [   4, 6028, 2448, 2419, 6021]]))

In [148]:
index_flat = faiss.IndexFlatL2(d)  
index_flat.add(feature_list_compressed)
D_flat, I_flat = index_flat.search(feature_list_compressed, k_nearest)
D_flat[:5], I_flat[:5]

(array([[3.9062500e-03, 1.5674359e+04, 1.7318316e+04, 2.2993758e+04,
         2.3135705e+04],
        [0.0000000e+00, 2.1488664e+04, 2.4966252e+04, 2.5156342e+04,
         2.6139074e+04],
        [0.0000000e+00, 3.6603906e+03, 3.8096055e+03, 4.0871562e+03,
         4.8052266e+03],
        [3.9062500e-03, 1.2588488e+04, 1.3350793e+04, 1.3417449e+04,
         1.4603053e+04],
        [0.0000000e+00, 2.2566000e+04, 2.3659629e+04, 2.3905688e+04,
         2.4714777e+04]], dtype=float32),
 array([[   0,    8,  168, 7939,    6],
        [   1,  149,  344,  213,  309],
        [   2,  138,  431,  157,  306],
        [   3,  426,  208,   75, 6222],
        [   4, 5403, 2562, 6021, 2533]]))

In [1113]:
compare(I_flat, feature_list_compressed, index, 'IndexIVFPQ (compressed)')
for nprobe, (D_list, I_list) in nprobe_test.items():
    print(f'Recall@5 for Custom IndexIVFPQ nprobe={nprobe}:', (I_list[:,1:6] == I_flat[:,[1]]).sum()/len(feature_list_compressed))

Recall@5 for IndexIVFPQ (compressed): 0.6674321959755031
Recall@5 for Custom IndexIVFPQ nprobe=1: 0.6917104111986002
Recall@5 for Custom IndexIVFPQ nprobe=2: 0.7362204724409449
Recall@5 for Custom IndexIVFPQ nprobe=3: 0.7469378827646544


### Implementation correctness

- Custom IVFPQ implementation looks right based on high Recall@5 (even higher than faiss)
- Increasing nprobe from 1 to 2 to 3 shows increasing number of true nearest neighbors found in top 5 neighbors (excluding self)
- Time to train and search is slow though

## Packing functions into a class to simplify method signatures

In [1381]:
from collections import defaultdict
import heapq
import operator

import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import euclidean_distances

class CustomIndexIVFPQ:
    
    BITS2DTYPE = {
        8: np.uint8,
        16: np.uint16,
    }
    
    def __init__(self,
                 d: int,
                 m: int,
                 nlist: int,
                 nbits: int,) -> None:
        """Custom IndexIVFPQ implementation.
        
        Parameters
        ----------
        d
            Dimensionality of the original vectors.
        m
            Number of segments.
        nlist
            Number of coarse centroids for IVF
        nbits
            Number of bits.
        """
        if d % m != 0:
            raise ValueError("d needs to be a multiple of m")

        if nbits not in CustomIndexIVFPQ.BITS2DTYPE:
            raise ValueError(f"Unsupported number of bits {nbits}")

        self.m = m
        self.nlist = nlist
        self.nprobe = 1
        self.k = 2**nbits
        self.d = d
        self.ds = d // m

        self.coarse_quantizer = KMeans(n_clusters=self.nlist, random_state=1)
        self.inverted_list = defaultdict(list)
        self.max_id = 0 # to start following batches of vector adding from the right index to prevent duplicate ids if in different IVF cell or overwriting of data if in same IVF cell
        self.codes_db = np.empty((0,m),dtype=CustomIndexIVFPQ.BITS2DTYPE[nbits]) # always cleared in jupyter for convenience
        
        self.fine_quantizers = [KMeans(n_clusters=self.k, random_state=1) for _ in range(m)]

        self.is_trained = False

        self.dtype = CustomIndexIVFPQ.BITS2DTYPE[nbits]
        self.dtype_orig = np.float32
        
    def fit_coarse_quantizer(self, feature_list):
        """Coarse quantizer to divide database vectors into various voronoi cells so search only probes nprobe closest
        
        Parameters
        ----------
        feature_list
            Data to be converted to residuals
        
        Returns
        -------
        feature_list_residual
            Residuals created by subtracting each vector with it's own coarse centroid
        """
        self.coarse_quantizer.fit(feature_list)
        feature_list_residual = feature_list - self.coarse_quantizer.cluster_centers_[self.coarse_quantizer.labels_]  # generate residual database vectors to be fine quantized
        return feature_list_residual
    
    def fit_fine_quantizer(self, feature_list_residual):
        """Fit m fine quantizers for each of m segments
        
        Parameters
        ----------
        feature_list_residual
            Residuals created by subtracting each vector with it's own coarse centroid
        """
        for i in range(self.m):
            X_i = feature_list_residual[:, i * self.ds : (i + 1) * self.ds]
            self.fine_quantizers[i].fit(X_i)
    
    def apply_coarse_quantizer(self, feature_list):
        """Find closest IVF centroid using coarse quantizer and get residuals
        
        Parameters
        ----------
        feature_list
            Raw Vectors to be assigned to coarse quantizer centroids
        
        Returns
        -------
        feature_list_residual
            Residuals to be quantized by fine quantizer after coarse quantization
        
        ivf_cells
            labels of which coarse centroids each vector goes into
        """
        ivf_cells = self.coarse_quantizer.predict(feature_list)
        feature_list_residual = feature_list - self.coarse_quantizer.cluster_centers_[ivf_cells]  # generate residual database vectors to be fine quantized
        return feature_list_residual, ivf_cells
            
    def quantize_residuals(self, feature_list_residual):
        """
        Fine quantization of residuals of both database and query vectors
        
        Parameters
        ----------
        feature_list_residual
            Residuals created by subtracting each vector with it's own coarse centroid
    
        Returns
        -------
        codes
            Quantized codes of residuals, shaped n, m  
        """ 
        n = len(feature_list_residual)
        codes = np.empty((n, self.m), dtype=self.dtype)  #Prevents automatic 'float64' causing IndexError: arrays used as indices must be of integer (or boolean) type

        for i in range(self.m):
            estimator = self.fine_quantizers[i]
            X_i = feature_list_residual[:, i * self.ds : (i + 1) * self.ds]
            codes[:, i] = estimator.predict(X_i)  # shape n number of vectors, m segments
            
        return codes
    
    def add_inverted_list(self, ivf_cells, codes):
        """
        Assign ids to cells and add codes to database
        
        Parameters
        ----------
        ivf_cells
            coarse quantized labels of vectors
        codes
            Quantized codes of residuals to be added to database of quantized vectors
        """
        for idx, coarse_center in enumerate(ivf_cells, start=self.max_id):  
            self.inverted_list[coarse_center].append(idx)
        
        self.max_id += len(codes) # update max_id so next addition to IVF don't duplicate id (if same different coarse_center) or overwrite data (if same coarse_center)
        self.codes_db = np.vstack([self.codes_db, codes])
    
    def distance_to_IVFcentroids(self, query):
        """
        Find distance of raw query to coarse centroids
        
        Parameters
        ----------
        query
            Raw query vector
        
        """
        query_distance_to_coarse_centroids = euclidean_distances(query, self.coarse_quantizer.cluster_centers_, squared=True)[0]
        nearest_inverted_keys = np.argsort(query_distance_to_coarse_centroids)[:self.nprobe]  # argsort gives index of closest coarse centroids, to be filtered by nprobe
        return nearest_inverted_keys 

    def generate_query_residual(self, query, current_cell):
        """Find closest IVF centroid and return residual of query from that centroid
        
        Parameters
        ----------
        query
            Raw query vector
        current_cell
            A particular coarse centroid explored during probing for IVF cells
        
        Returns
        -------
        """
        query_residual = query - self.coarse_quantizer.cluster_centers_[current_cell]  # generate residual query to be compared against all quantized residuals
        
        return query_residual
    
    def compute_distance_table(self, query_residual):
        """Distance table per coarse centroid for reuse by all quantized residual vectors in same cell 

        Parameters
        ----------
        query_residual
            Residual of query vector
            
        Returns
        -------
        distance_table 
            Table of distances from each query vector to all k clusters, for each segment, to be used by all database vectors in same coarse centroid
        """
        distance_table = np.empty((self.m, self.k), dtype=self.dtype_orig)  # shape m segments, distance to k clusters 

        for i in range(self.m):
            X_i = query_residual[:, i * self.ds : (i + 1) * self.ds]
            centers = self.fine_quantizers[i].cluster_centers_  # (k, ds)
            distance_table[i, :] = euclidean_distances(X_i, centers, squared=True)
        return distance_table
    
    def filter_residual_vectors(self, current_cell):
        """Identify only relevant vectors in same cell as query to compute distances for
        
        Parameters
        ----------
        current_cell
            particular coarse centroid explored during probing for IVF cells
        
        Returns
        -------
        filtered_result
            Filtered codes so search only calculates their distances
        filtered_ids
            Filtered ids used for indicating which vectors are being searched
        """
    
        filtered_ids = self.inverted_list[current_cell]
        filtered_result = self.codes_db[filtered_ids]
        
        return filtered_result, filtered_ids
    
    def calculate_distances(self, filtered_result, distance_table):
        """Calculate distance of each database vector with quantized query vector
        
        Parameters
        ----------
        filtered_result
            Filtered codes for calculating distances with their coarse centroid 
        distance_table
            Table of distances from each query vector to all k clusters, for each segment, to be used by all database vectors in same coarse centroid
        
        Returns
        -------
        distances
            Distance of each database vector with quantized query vector
        """
        distances = np.zeros(len(filtered_result), dtype=self.dtype_orig)

        for i in range(m):
            distances += distance_table[i, filtered_result[:, i]]
        
        return distances
    
    def find_smallest_k(self, distances, filtered_ids, k_nearest):
        """Find nearest k neighbors (including self)
        
        Parameters
        ----------
        distances
            Distance of each database vector with quantized query vector
        filtered_ids
            ids of vectors
        k_nearest
            Number of approximate neighbors
        
        
        Returns
        -------
        D
            K nearest distances per query vector
        I
            Indices of the K nearest distances per query vector
        """
        distance_id = zip(distances,filtered_ids)
        D, I = zip(*heapq.nsmallest(k_nearest, distance_id, operator.itemgetter(0)))
        
        return D, I

    def train(self, feature_list: np.ndarray) -> None:
        """Train the index given data
        
        Parameters
        ----------
        feature_list
            Array of shape `(n, d)` and dtype `float32`.
        """
        feature_list_residual = self.fit_coarse_quantizer(feature_list)
        self.fit_fine_quantizer(feature_list_residual)
        
        self.is_trained = True

    def add(self, feature_list: np.ndarray) -> None:
        """Add vectors to the database (their encoded versions).
        
        Parameters
        ----------
        feature_list
            Array of shape `(n_codes, d)` of dtype `np.float32`.
        Raises
        ------
        ValueError
            Cannot add data if quantizers not trained
        """
        if not self.is_trained:
            raise ValueError("Both coarse and fine quantizers need to be trained first.")
    
        feature_list_residual, ivf_cells = self.apply_coarse_quantizer(feature_list)
        codes = self.quantize_residuals(feature_list_residual)
        self.add_inverted_list(ivf_cells, codes)

    def search(self, query: np.ndarray, k_nearest: int) -> tuple:
        """Search for k nearest neighbors
        
        Parameters
        ----------
        query
            Raw query vector
        k_nearest
            Number of approximate neighbors
        
        
        Returns
        -------
        D
            K nearest distances per query vector
        I
            Indices of the K nearest distances per query vector
            
        Raises
        ------
        ValueError
            Cannot add data if quantizers not trained
            Cannot search database if it's empty
        
        """
        if not self.is_trained:
            raise ValueError("Both coarse and fine quantizers need to be trained first.")
        
        if self.codes_db.size == 0:
            raise ValueError("No codes detected. You need to run `add` first")
        
        nearest_inverted_keys = self.distance_to_IVFcentroids(query)
        
        nprobe_distances = np.array([], dtype=self.dtype_orig)
        nprobe_filtered_ids = np.array([], dtype=np.uint64)
        
        for current_cell in nearest_inverted_keys:
            query_residual = self.generate_query_residual(query, current_cell)
            distance_table = self.compute_distance_table(query_residual)
            filtered_result, filtered_ids = self.filter_residual_vectors(current_cell)
            distances = self.calculate_distances(filtered_result, distance_table)
            
            nprobe_distances = np.append(nprobe_distances, distances)
            nprobe_filtered_ids = np.append(nprobe_filtered_ids, filtered_ids)
            
        D, I = self.find_smallest_k(nprobe_distances, nprobe_filtered_ids, k_nearest)
    
        # some will return < k neighbors, need to pad on right to form rectangular result array
        # assigning -1 into uint8 causes 18446744073709551615, ok for evaluation as long as doesn't match a ground truth index
        return np.pad(D,pad_width=(0,k_nearest-len(D)),constant_values=-1), np.pad(I,pad_width=(0,k_nearest-len(I)),constant_values=-1) 


In [1384]:
d = 128
m = 8
nlist = 100
nbits = 8
custom_ivfpq = CustomIndexIVFPQ(d,m,nlist,nbits)

custom_ivfpq.train(feature_list_compressed)

/Users/hanqi/code/ahrefs/venv/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Users/hanqi/code/ahrefs/venv/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Users/hanqi/code/ahrefs/venv/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Users/hanqi/code/ahrefs/venv/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.wa

In [1385]:
custom_ivfpq.inverted_list
custom_ivfpq.codes_db

defaultdict(list, {})

array([], shape=(0, 8), dtype=uint8)

In [1386]:
custom_ivfpq.add(feature_list_compressed)

In [1387]:
k_nearest = 6

nprobe_test = {}
# nprobes_to_test = range(1,4)
nprobes_to_test = [1]

for nprobe in nprobes_to_test:
    D_list = []
    I_list = []
    for query in feature_list_compressed:
        D, I = custom_ivfpq.search(query.reshape(1,-1),   #sklearn euclidean_distances needs 2D
                                   k_nearest)
        D_list.append(D)
        I_list.append(I)
        
    D_list = np.array(D_list, dtype=np.float32)
    I_list = np.array(I_list, dtype=np.uint64)
    
    nprobe_test[nprobe] = (D_list, I_list)

In [1286]:
I_list = nprobe_test[1][1]
print(f'Recall@5 for Custom IndexIVFPQ nprobe={nprobe}:', (I_list[:,1:6] == I_flat[:,[1]]).sum()/len(feature_list_compressed))

Recall@5 for Custom IndexIVFPQ nprobe=1: 0.6917104111986002


# Refactoring Custom IndexIVFPQ to switch both quantizers from sklearn to faiss.Kmeans
- Kmeans is too slow to train (40 secs on 9k caltech)

**Identifying API Changes** (preparing for find and replace)

|      Sklearn      |  Faiss |
|:-----------------:|-------:|
|   model.fit(x) | model.train(x) |
|   model.predict(x)|model.assign(x)[1] |
|   model.labels_|   model.assign(x)[1] |
|   model.cluster_centers_|  model.centroids |

## Proving shape equivalence between sklearn and faiss

In [100]:
import faiss
from sklearn.cluster import KMeans

np.random.seed(1)
x = np.random.random((10000,100))
ncentroids = 128
niter = 20
verbose = True
d = x.shape[1]

model = KMeans(n_clusters=ncentroids, random_state=1)
model.fit(x)
print('Kmeans predict shape: ', model.predict(x).shape)
print('Kmeans labels shape: ', model.labels_.shape)
print('Kmeans cluster centers shape: ', model.cluster_centers_.shape)

kmeans = faiss.Kmeans(d, ncentroids, verbose=verbose, seed=1)
kmeans.train(x) # model.fit() 

print('faiss predict shape: ', kmeans.assign(x)[1].shape) # model.labels_ or model.predict(x)
print('faiss cluster centers shape ', kmeans.centroids.shape) # model.cluster_centers_



/Users/hanqi/code/ahrefs/venv/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=128, random_state=1)

Kmeans predict shape:  (10000,)
Kmeans labels shape:  (10000,)
Kmeans cluster centers shape:  (128, 100)
Clustering 10000 points in 100D to 128 clusters, redo 1 times, 25 iterations
  Preprocessing in 0.00 s
  Iteration 24 (0.21 s, search 0.19 s): objective=74839.4 imbalance=1.117 nsplit=0       


74839.4453125

faiss predict shape:  (10000,)
faiss cluster centers shape  (128, 100)


In [151]:
from collections import defaultdict
import heapq
import operator

import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import euclidean_distances
import faiss


class CustomIndexIVFPQ:
    BITS2DTYPE = {
        8: np.uint8,
        16: np.uint16,
    }

    def __init__(
        self,
        d: int,
        m: int,
        nlist: int,
        nbits: int,
    ) -> None:
        """Custom IndexIVFPQ implementation.

        Parameters
        ----------
        d
            Dimensionality of the original vectors.
        m
            Number of segments.
        nlist
            Number of coarse centroids for IVF
        nbits
            Number of bits.
        """
        if d % m != 0:
            raise ValueError("d needs to be a multiple of m")

        if nbits not in CustomIndexIVFPQ.BITS2DTYPE:
            raise ValueError(f"Unsupported number of bits {nbits}")

        self.m = m
        self.nlist = nlist
        self.nprobe = 1
        self.k = 2**nbits
        self.d = d
        self.ds = d // m

        # self.coarse_quantizer = KMeans(n_clusters=self.nlist, random_state=1)
        self.coarse_quantizer = faiss.Kmeans(d, nlist, seed=1)
        self.inverted_list = defaultdict(list)
        self.max_id = 0  # to start following batches of vector adding from the right index to prevent duplicate ids if in different IVF cell or overwriting of data if in same IVF cell
        self.codes_db = np.empty(
            (0, m), dtype=CustomIndexIVFPQ.BITS2DTYPE[nbits]
        ) 

        self.fine_quantizers = [
            # KMeans(n_clusters=self.k, random_state=1) for _ in range(m)
            faiss.Kmeans(self.ds, self.k, seed=1) for _ in range(m)
        ]

        self.is_trained = False

        self.dtype = CustomIndexIVFPQ.BITS2DTYPE[nbits]
        self.dtype_orig = np.float32

    def fit_coarse_quantizer(self, feature_list):
        """Coarse quantizer to divide database vectors into various voronoi cells so search only probes nprobe closest

        Parameters
        ----------
        feature_list
            Data to be converted to residuals

        Returns
        -------
        feature_list_residual
            Residuals created by subtracting each vector with it's own coarse centroid
        """
        self.coarse_quantizer.train(feature_list)
        feature_list_residual = (
            feature_list
            - self.coarse_quantizer.centroids[self.coarse_quantizer.assign(feature_list)[1]]
        )  # generate residual database vectors to be fine quantized
        return feature_list_residual

    def fit_fine_quantizer(self, feature_list_residual):
        """Fit m fine quantizers for each of m segments

        Parameters
        ----------
        feature_list_residual
            Residuals created by subtracting each vector with it's own coarse centroid
        """
        for i in range(self.m):
            X_i = feature_list_residual[:, i * self.ds : (i + 1) * self.ds]
            self.fine_quantizers[i].train(X_i)

    def apply_coarse_quantizer(self, feature_list):
        """Find closest IVF centroid using coarse quantizer and get residuals

        Parameters
        ----------
        feature_list
            Raw Vectors to be assigned to coarse quantizer centroids

        Returns
        -------
        feature_list_residual
            Residuals to be quantized by fine quantizer after coarse quantization

        ivf_cells
            labels of which coarse centroids each vector goes into
        """
        ivf_cells = self.coarse_quantizer.assign(feature_list)[1]
        feature_list_residual = (
            feature_list - self.coarse_quantizer.centroids[ivf_cells]
        )  # generate residual database vectors to be fine quantized
        return feature_list_residual, ivf_cells

    def quantize_residuals(self, feature_list_residual):
        """
        Fine quantization of residuals of both database and query vectors

        Parameters
        ----------
        feature_list_residual
            Residuals created by subtracting each vector with it's own coarse centroid

        Returns
        -------
        codes
            Quantized codes of residuals, shaped n, m
        """
        n = len(feature_list_residual)
        codes = np.empty(
            (n, self.m), dtype=self.dtype
        )  # Prevents automatic 'float64' causing IndexError: arrays used as indices must be of integer (or boolean) type

        for i in range(self.m):
            estimator = self.fine_quantizers[i]
            X_i = feature_list_residual[:, i * self.ds : (i + 1) * self.ds]
            codes[:, i] = estimator.assign(
                X_i)[1]  # shape n number of vectors, m segments

        return codes

    def add_inverted_list(self, ivf_cells, codes):
        """
        Assign ids to cells and add codes to database

        Parameters
        ----------
        ivf_cells
            coarse quantized labels of vectors
        codes
            Quantized codes of residuals to be added to database of quantized vectors
        """
        for idx, coarse_center in enumerate(ivf_cells, start=self.max_id):
            self.inverted_list[coarse_center].append(idx)

        self.max_id += len(
            codes
        )  # update max_id so next addition to IVF don't duplicate id (if same different coarse_center) or overwrite data (if same coarse_center)
        self.codes_db = np.vstack([self.codes_db, codes])

    def distance_to_IVFcentroids(self, query):
        """
        Find distance of raw query to coarse centroids

        Parameters
        ----------
        query
            Raw query vector

        """
        query_distance_to_coarse_centroids = euclidean_distances(
            query, self.coarse_quantizer.centroids, squared=True
        )[0]
        nearest_inverted_keys = np.argsort(query_distance_to_coarse_centroids)[
            : self.nprobe
        ]  # argsort gives index of closest coarse centroids, to be filtered by nprobe
        return nearest_inverted_keys

    def generate_query_residual(self, query, current_cell):
        """Find closest IVF centroid and return residual of query from that centroid

        Parameters
        ----------
        query
            Raw query vector
        current_cell
            A particular coarse centroid explored during probing for IVF cells

        Returns
        -------
        """
        query_residual = (
            query - self.coarse_quantizer.centroids[current_cell]
        )  # generate residual query to be compared against all quantized residuals

        return query_residual

    def compute_distance_table(self, query_residual):
        """Distance table per coarse centroid for reuse by all quantized residual vectors in same cell

        Parameters
        ----------
        query_residual
            Residual of query vector

        Returns
        -------
        distance_table
            Table of distances from each query vector to all k clusters, for each segment, to be used by all database vectors in same coarse centroid
        """
        distance_table = np.empty(
            (self.m, self.k), dtype=self.dtype_orig
        )  # shape m segments, distance to k clusters

        for i in range(self.m):
            X_i = query_residual[:, i * self.ds : (i + 1) * self.ds]
            centers = self.fine_quantizers[i].centroids  # (k, ds)
            distance_table[i, :] = euclidean_distances(X_i, centers, squared=True)
        return distance_table

    def filter_residual_vectors(self, current_cell):
        """Identify only relevant vectors in same cell as query to compute distances for

        Parameters
        ----------
        current_cell
            particular coarse centroid explored during probing for IVF cells

        Returns
        -------
        filtered_result
            Filtered codes so search only calculates their distances
        filtered_ids
            Filtered ids used for indicating which vectors are being searched
        """

        filtered_ids = self.inverted_list[current_cell]
        filtered_result = self.codes_db[filtered_ids]

        return filtered_result, filtered_ids

    def calculate_distances(self, filtered_result, distance_table):
        """Calculate distance of each database vector with quantized query vector

        Parameters
        ----------
        filtered_result
            Filtered codes for calculating distances with their coarse centroid
        distance_table
            Table of distances from each query vector to all k clusters, for each segment, to be used by all database vectors in same coarse centroid

        Returns
        -------
        distances
            Distance of each database vector with quantized query vector
        """
        distances = np.zeros(len(filtered_result), dtype=self.dtype_orig)

        for i in range(m):
            distances += distance_table[i, filtered_result[:, i]]

        return distances

    def find_smallest_k(self, distances, filtered_ids, k_nearest):
        """Find nearest k neighbors (including self)

        Parameters
        ----------
        distances
            Distance of each database vector with quantized query vector
        filtered_ids
            ids of vectors
        k_nearest
            Number of approximate neighbors


        Returns
        -------
        D
            K nearest distances per query vector
        I
            Indices of the K nearest distances per query vector
        """
        distance_id = zip(distances, filtered_ids)
        D, I = zip(*heapq.nsmallest(k_nearest, distance_id, operator.itemgetter(0)))

        return D, I

    def train(self, feature_list: np.ndarray) -> None:
        """Train the index given data

        Parameters
        ----------
        feature_list
            Array of shape `(n, d)` and dtype `float32`.
        """
        feature_list_residual = self.fit_coarse_quantizer(feature_list)
        self.fit_fine_quantizer(feature_list_residual)

        self.is_trained = True

    def add(self, feature_list: np.ndarray) -> None:
        """Add vectors to the database (their encoded versions).

        Parameters
        ----------
        feature_list
            Array of shape `(n_codes, d)` of dtype `np.float32`.
        Raises
        ------
        ValueError
            Cannot add data if quantizers not trained
        """
        if not self.is_trained:
            raise ValueError(
                "Both coarse and fine quantizers need to be trained first."
            )

        feature_list_residual, ivf_cells = self.apply_coarse_quantizer(feature_list)
        codes = self.quantize_residuals(feature_list_residual)
        self.add_inverted_list(ivf_cells, codes)

    def search(self, query: np.ndarray, k_nearest: int) -> tuple:
        """Search for k nearest neighbors

        Parameters
        ----------
        query
            Raw query vector
        k_nearest
            Number of approximate neighbors


        Returns
        -------
        D
            K nearest distances per query vector
        I
            Indices of the K nearest distances per query vector

        Raises
        ------
        ValueError
            Cannot add data if quantizers not trained
            Cannot search database if it's empty

        """
        if not self.is_trained:
            raise ValueError(
                "Both coarse and fine quantizers need to be trained first."
            )

        if self.codes_db.size == 0:
            raise ValueError("No codes detected. You need to run `add` first")

        nearest_inverted_keys = self.distance_to_IVFcentroids(query)

        nprobe_distances = np.array([], dtype=self.dtype_orig)
        nprobe_filtered_ids = np.array([], dtype=np.uint64)

        for current_cell in nearest_inverted_keys:
            query_residual = self.generate_query_residual(query, current_cell)
            distance_table = self.compute_distance_table(query_residual)
            filtered_result, filtered_ids = self.filter_residual_vectors(current_cell)
            distances = self.calculate_distances(filtered_result, distance_table)

            nprobe_distances = np.append(nprobe_distances, distances)
            nprobe_filtered_ids = np.append(nprobe_filtered_ids, filtered_ids)

        D, I = self.find_smallest_k(nprobe_distances, nprobe_filtered_ids, k_nearest)
        
        # some will return < k neighbors, need to pad on right to form rectangular result array
        # assigning -1 into uint8 causes 18446744073709551615, ok for evaluation as long as doesn't match a ground truth index
        return np.pad(D,pad_width=(0,k_nearest-len(D)),constant_values=-1), np.pad(I,pad_width=(0,k_nearest-len(I)),constant_values=-1) 


## Verifying correctness of refactoring
- Recall@5 for coarse quantizer using faiss instead of sklean kmeans dropped to 0.66 from 0.69
- Training speed improved from 40 seconds to < 2seconds

In [152]:
d = 128
m = 8
nlist = 100
nbits = 8
custom_ivfpq = CustomIndexIVFPQ(d,m,nlist,nbits)

custom_ivfpq.train(feature_list_compressed)

WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 9144 points to 256 centroids: please provide at least 9984 training points


In [153]:
custom_ivfpq.add(feature_list_compressed)

In [154]:
k_nearest = 6

nprobe_test = {}
# nprobes_to_test = range(1,4)
nprobes_to_test = [1]

for nprobe in nprobes_to_test:
    D_list = []
    I_list = []
    
    for query in feature_list_compressed:
        D, I = custom_ivfpq.search(query.reshape(1,-1),   #sklearn euclidean_distances needs 2D
                                k_nearest)
        D_list.append(D)   
        I_list.append(I)
        
    D_list = np.array(D_list, dtype=np.float32)
    I_list = np.array(I_list, dtype=np.uint64)
    
    nprobe_test[nprobe] = (D_list, I_list)

In [155]:
I_list = nprobe_test[1][1]
print(f'Recall@5 for Custom IndexIVFPQ nprobe={nprobe}:', (I_list[:,1:6] == I_flat[:,[1]]).sum()/len(feature_list_compressed))

Recall@5 for Custom IndexIVFPQ nprobe=1: 0.665573053368329
